In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from continuousEnv import ContinuousEnv, EnvArgs
import numpy as np
import random
from DNC_PPO import PPO
import csv
import matplotlib.pyplot as plt#matplotlib inline
import math


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:

def get_bandwidth(main_path):
    file_list = os.listdir(main_path)
    bandwidth = {}
    count = 0
    bandwidth[0] = []
    for f in file_list:
        if (f.startswith("report_foot_") == True and count < 5):
            with open(main_path + '/' + f, 'r') as file_to_read:
                while True:
                    lines = file_to_read.readline()
                    if not lines:
                        break
                    item = [i for i in lines.split()]
                    bandwidth[count].append(float(item[-2])/1000/1000)
            count += 1
            bandwidth[count] = []
    return bandwidth

# def main():
#     train_ppo()

# set the experiment environment
user_num = 100
his_len = 5
info_num = 1
main_path = "./Dataset"
bandwidth = get_bandwidth(main_path)
C = np.array([18,20,22,24,26]).astype("float")
D = np.array([0.08, 0.06, 0.07, 0.06, 0.09]).astype("float")
alpha = np.array([1,1,1,1,1]) / 50
tau = 2
epsilon = 5
env_args = EnvArgs(user_num, his_len, info_num, bandwidth, C, D, alpha, tau, epsilon)
env = ContinuousEnv(env_args)

# set the DRL agent
A_DIM, S_DIM = user_num, user_num * his_len * info_num
BATCH = 20
A_UPDATE_STEPS = 5
C_UPDATE_STEPS = 5
HAVE_TRAIN = False
A_LR = 0.00003
C_LR = 0.00003
v_s = np.zeros(user_num)
GAMMA = 0.95
EP_MAX = 500
EP_LEN = 400
dec = 0.3
action = np.zeros(user_num)
ppo = PPO(S_DIM, A_DIM, BATCH, A_UPDATE_STEPS, C_UPDATE_STEPS, HAVE_TRAIN, 0)

# define csvfiles for writing results
Algs = "dnc"
csvFile1 = open("test-lambda=0.5-Rewards_" + Algs + "_" + str(user_num) + ".csv", 'w', newline='')
writer1 = csv.writer(csvFile1)
csvFile2 = open("test-lambda=0.5-Actions_" + Algs + "_" + str(user_num) + ".csv", 'w', newline='')
writer2 = csv.writer(csvFile2)
csvFile3 = open("test-lambda=0.5-Aloss_" + Algs + "_" + str(user_num) + ".csv", 'w', newline='')
writer3 = csv.writer(csvFile3)
csvFile4 = open("test-lambda=0.5-Closs_" + Algs + "_" + str(user_num) + ".csv", 'w', newline='')
writer4 = csv.writer(csvFile4)

rewards = []
actions = []
closses = []
alosses = []
Ts = []
Es = []
cur_state = env.reset()
for ep in range(EP_MAX):
#     cur_state = env.reset()
    if ep % 50 == 0:
        dec = dec * 0.95
        A_LR = A_LR * 0.85
        C_LR = C_LR * 0.85
    buffer_s = []
    buffer_a = []
    buffer_r = []
    sum_reward = 0
    sum_action = 0
    sum_closs = 0
    sum_aloss = 0
    sum_T = 0
    sum_E = 0
    for t in range(EP_LEN):
        action = ppo.choose_action(cur_state.reshape(-1,S_DIM), dec)
#         action = np.random.random(np.shape(action))
        next_state, reward, T, E= env.step(1 + action * 1)
#         print(action,T,E)

        sum_reward += reward
        sum_action += action
        sum_T += T
        sum_E += E
        buffer_a.append(action.copy())
        buffer_s.append(cur_state.reshape(-1,S_DIM).copy())
        buffer_r.append(reward)

        cur_state = next_state
        # update ppo
        if (t + 1) % BATCH == 0:
            discounted_r = np.zeros(len(buffer_r), 'float32')
            v_s = ppo.get_v(next_state.reshape(-1, S_DIM))
            running_add = v_s

            for rd in reversed(range(len(buffer_r))):
                running_add = running_add * GAMMA + buffer_r[rd]
                discounted_r[rd] = running_add

            discounted_r = discounted_r[np.newaxis, :]
            discounted_r = np.transpose(discounted_r)
            if HAVE_TRAIN == False:
                closs, aloss = ppo.update(np.vstack(buffer_s), np.vstack(buffer_a), discounted_r, dec, A_LR, C_LR, ep)
                sum_closs += closs
                sum_aloss += aloss
    if ep % 10 == 0:
        print('instant ep:', ep)
        print("instant reward:", reward)
        print("instant action:", action)
        rewards.append(sum_reward / EP_LEN)
        actions.append(sum_action / EP_LEN)
        closses.append(sum_closs / EP_LEN)
        alosses.append(sum_aloss / EP_LEN)
        Ts.append(sum_T / EP_LEN)
        Es.append(sum_E / EP_LEN)
        print("average reward:", sum_reward / EP_LEN)
        print("average T:", sum_T / EP_LEN)
        print("average E:", sum_E / EP_LEN)
        print("average action:", sum_action / EP_LEN)
        print("average closs:", sum_closs / EP_LEN)
        print("average aloss:", sum_aloss / EP_LEN)

plt.plot(rewards)
plt.show()
writer1.writerow(-rewards * 10)
writer1.writerow(Ts)
writer1.writerow(Es)
for i in range(len(actions)):
    writer2.writerow(actions[i])
writer3.writerow(closses)
writer4.writerow(alosses)
csvFile1.close()
csvFile2.close()
csvFile3.close()
csvFile4.close()


# if __name__ == '__main__':
#     main()


instant ep: 0
instant reward: -0.9588143330598321
instant action: [0.5144526  0.5491903  0.46010908 0.7141957  0.5758558  0.43307352
 0.16639632 0.43684402 0.48745167 0.40726894 0.23268044 0.4393447
 0.49196273 0.4572337  0.64433014 0.5489645  0.3844334  0.42108744
 0.6358967  0.31626755 0.4496011  0.4805015  0.50564235 0.6327112
 0.42475337 0.47169855 0.5433571  0.626154   0.68254507 0.5543096
 0.68676317 0.6847447  0.39263004 0.76552904 0.48003602 0.54817176
 0.4019747  0.73884404 0.2711258  0.3788245  0.52287644 0.66762537
 0.6886092  0.5380451  0.30556282 0.52797556 0.5074983  0.418523
 0.558712   0.37237456 0.24409801 0.74034286 0.27022052 0.5408663
 0.39497185 0.3513199  0.30951482 0.67738557 0.33285624 0.45762178
 0.500098   0.19141534 0.5217043  0.64624125 0.38350242 0.5075825
 0.38669193 0.5567454  0.5817051  0.58511287 0.7032964  0.5780612
 0.3918283  0.75241756 0.47537807 0.61574775 0.7100515  0.39673108
 0.3491882  0.6014289  0.39634913 0.43057263 0.61409974 0.6474752
 0.36

instant ep: 40
instant reward: -0.9047934391690358
instant action: [0.46212417 0.4146105  0.49887675 0.60875356 0.7138952  0.40219507
 0.58531624 0.30520713 0.28667206 0.37119386 0.8662906  0.3584087
 0.6906537  0.769645   0.6867543  0.44583642 0.38055694 0.4851015
 0.79356813 0.43579164 0.6345974  0.7839403  0.44847977 0.5049768
 0.52651256 0.5244058  0.48732546 0.21552166 0.6198009  0.6205403
 0.4692002  0.5208485  0.61252195 0.49955478 0.42334166 0.39653793
 0.34171206 0.29452372 0.71006984 0.6389096  0.45048746 0.2734722
 0.57156664 0.35458815 0.1808602  0.70393896 0.2763939  0.45506078
 0.46388942 0.39512318 0.58949023 0.22374313 0.3960432  0.41162312
 0.6695931  0.15494752 0.42452338 0.53385967 0.48917598 0.6490925
 0.3791781  0.41152748 0.55637443 0.217487   0.50892204 0.6758861
 0.35921806 0.33555102 0.50401855 0.5843482  0.21620885 0.63107747
 0.5147677  0.19338247 0.23661733 0.2796721  0.40644377 0.25277254
 0.5157605  0.47662222 0.6875327  0.6963108  0.60125256 0.31462753
 0

In [ ]:
plt.plot(rewards[2:])
plt.show()

In [ ]:
writer1.writerow(rewards * (-10))
writer1.writerow(Ts)
writer1.writerow(Es)
for i in range(len(actions)):
    writer2.writerow(actions[i])
writer3.writerow(closses)
writer4.writerow(alosses)
csvFile1.close()
csvFile2.close()
csvFile3.close()
csvFile4.close()
tmp = []
fig = plt.figure()
for i in range(len(alosses)):
    tmp.append(-sum(alosses[0:0+i+1])/len(alosses[0:0+i+1]))
plt.plot(tmp)
plt.show()   
fig.savefig("aloss.png")
print(tmp)